# SHO Fitter on the noisy data

In [6]:
import sys

sys.path.append("../../")
sys.path.append("/home/ferroelectric/m3_learning/m3_learning/src")

In [9]:
%load_ext autoreload
%autoreload 2

import numpy as np
from m3_learning.be.dataset import BE_Dataset
from m3_learning.viz.printing import printer
from m3_learning.be.nn import SHO_fit_func_nn, SHO_Model


# from m3_learning.be.dataset import BE_Dataset
printing = printer(basepath = './../../../Figures/2023_rapid_fitting/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# import sys
# sys.path.append('../../')
# sys.path.append('/home/ferroelectric/m3_learning/m3_learning/src')


In [2]:
# import matplotlib.pyplot as plt
# import numpy as np
# import h5py
# import time
# import pandas as pd
# import os
# import shutil
# import gc
# import gdown

# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader

# from scipy.signal import resample
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split 

# from src.m3_learning.optimizers.AdaHessian import AdaHessian
# from src.m3_learning.nn.SHO_fitter.SHO import SHO_fit_func_torch
# from src.m3_learning.be.processing import convert_amp_phase, SHO_Fitter, SHO_fit_to_array
# from src.m3_learning.nn.random import random_seed
# from src.m3_learning.util.preprocessing import global_scaler
# from src.m3_learning.viz.layout import layout_fig

# from src.m3_learning.be.util import print_be_tree

# from src.m3_learning.viz.printing import printer
# from src.m3_learning.viz.layout import combine_lines, labelfigs


# printing = printer(basepath='./figures/')

In [3]:
# gdown.download(
#     "https://drive.google.com/uc?export=download&id=1Q2Qo_1VGlCsVOTjQpZlE5tjoIV1etVe2",
#     path + "data_file_google.h5",
#     quiet=False,
#     resume=True,
# )

## Loads data

In [4]:
# # Sets path to file
# path = r"./"

# # Opens the data file
# h5_f = h5py.File(path + "data_file.h5", "r+")

# # number of pixels in the image
# num_pix = h5_f["Measurement_000"].attrs["num_pix"]

# num_pix_1d = int(np.sqrt(num_pix))

# # Frequency Vector in Hz
# frequency_bin = h5_f["Measurement_000"]["Channel_000"]["Bin_Frequencies"][:]

# # extracting spectroscopic values
# spectroscopic_values = h5_f['Measurement_000']['Channel_000']['Spectroscopic_Values']

# # number of DC voltage steps
# voltage_steps = h5_f["Measurement_000"].attrs["num_udvs_steps"]

# # Resampled frequency vector
# wvec_freq = resample(frequency_bin, 80)

# # get raw data (real and imaginary combined)
# raw_data = h5_f["Measurement_000"]["Channel_000"]["Raw_Data"]
# raw_data_resampled = resample(np.array(h5_f["Measurement_000"]["Channel_000"]["Raw_Data"]).reshape(-1, 165), 80, axis=1)

# # conversion of raw data (both resampled and full)
# amp, phase = convert_amp_phase(raw_data)
# amp_resample, phase_resample = convert_amp_phase(raw_data_resampled)

# scaled_data = h5_f["Measurement_000"]["Channel_000"]['complex']['scaled_data'][:]
# real_resample = h5_f["Measurement_000"]["Channel_000"]['complex']['real_resample'][:]
# imag_resample = h5_f["Measurement_000"]["Channel_000"]['complex']['imag_resample'][:]

# # scale the real component of input data
# scaler_real = global_scaler()
# scaled_data_real = scaler_real.fit_transform(real_resample).reshape(-1, 80)

# # scale the imaginary component of input data
# scaler_imag = global_scaler()
# scaled_data_imag = scaler_imag.fit_transform(imag_resample).reshape(-1, 80)

In [17]:
# Specify the filename and the path to save the file
filename = "data_raw copy.h5"
save_path = "./../../../Data/2023_rapid_fitting"


data_path = save_path + "/" + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path, resample_bins=80, SHO_fit_func_NN=SHO_fit_func_nn)

# print the contents of the file
dataset.print_be_tree()

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Raw_Data_Reshaped
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
    ├ original_data_2STD
    ├ raw_data_resampled
├ Raw_Data-SHO_Fit_000
  --------------------
  ├ Fit
  ├ Guess
  ├ SHO_LSQF
  ├ Spectroscopic_Indices
  ├ Spec

In [ ]:
# # create a list for parameters
# fit_results_list = SHO_fit_to_array(h5_f["Measurement_000"]["Channel_000"]["Raw_Data-SHO_Fit_000"]["Fit"])

# # flatten parameters list into numpy array
# fit_results_list = np.array(fit_results_list).reshape(num_pix, voltage_steps, 5)

# # exclude the R2 parameter
# params = fit_results_list.reshape(-1, 5)[:, 0:4]

# # scale the parameters (now takes only 4 parameters, excluding the R2)
# params_scaler = StandardScaler()
# scaled_params = params_scaler.fit_transform(fit_results_list.reshape(-1, 5)[:, 0:4])

In [12]:
dataset.original_data.shape

(3600, 63360)

In [13]:
np.std(dataset.original_data)

0.0038833667

In [14]:
self = dataset

def generate_noisy_data(self, noise_multipliers = 2, noise_STD = None):
    
    if noise_STD is None:
        noise_STD = np.std(self.original_data)
        
    noise_level = noise_STD * noise_multipliers
    
    noise_real = np.random.uniform(-1*noise_level , noise_level, (3600, 63360))
    noise_imag = np.random.uniform(-1*noise_level, noise_level, (3600, 63360))
    noise = noise_real+noise_imag*1.0j  
    return self.original_data + noise

In [15]:
out = generate_noisy_data(self)

In [16]:
self.data_writer("Measurement_000/Channel_000", "original_data_2STD",out)

In [ ]:
# lowest_noise_real = -1.0 * 0.0026878386 # 2.6e-3
# highest_noise_real = 1.0 * 0.0026878386
# lowest_noise_imag = -1.0 * 0.0027575183
# highest_noise_imag = 1.0 * 0.0027575183

# noise_real = np.random.uniform(lowest_noise_real, highest_noise_real, (3600, 63360))
# noise_imag = np.random.uniform(lowest_noise_imag, highest_noise_imag, (3600, 63360))
# noise = noise_real+noise_imag*1.0j

# noise_levels = ['2.0', '4.0', '7.0']

In [ ]:
# all computations were pre-computed on A100 40GB NHI GPU
# set to False if you want to recompute
use_pre_computed_vars = False
use_pre_trained_models = False

In [ ]:
print_be_tree('./data_file_noise_7.0.h5')

### Copies initial dataset for the later use

In [ ]:
if not use_pre_computed_vars:
  # close the current file to perform a copy
  h5_f.close()
  for nl in noise_levels:
    shutil.copy('data_file_raw.h5', f'data_file_noise_{nl}.h5')
    time.sleep(3)

In [ ]:
for noise_level in noise_levels:
  
  with h5py.File(f'data_file_noise_{noise_level}.h5', 'r+') as h5_f_noise:
    data_pointer = h5_f_noise['Measurement_000']['Channel_000']['Raw_Data']
    raw_data_noise = np.array(h5_f_noise['Measurement_000']['Channel_000']['Raw_Data'])
    raw_data_noise = raw_data_noise + noise * float(noise_level) 
    data_pointer[...] = raw_data_noise
    del h5_f_noise['Measurement_000']['Channel_000']['Raw_Data-SHO_Fit_000']

## Builds the PyTorch model

In [ ]:
class SHO_Model(nn.Module):
    def __init__(self):
        super().__init__()

        # Input block of 1d convolution
        self.hidden_x1 = nn.Sequential(
            nn.Conv1d(in_channels=2, out_channels=8, kernel_size=7),
            nn.SELU(),
            nn.Conv1d(in_channels=8, out_channels=6, kernel_size=7),
            nn.SELU(),
            nn.Conv1d(in_channels=6, out_channels=4, kernel_size=5),
            nn.SELU(),
        )

        # fully connected block
        self.hidden_xfc = nn.Sequential(
            nn.Linear(256, 20),
            nn.SELU(),
            nn.Linear(20, 20),
            nn.SELU(),
        )

        # 2nd block of 1d-conv layers
        self.hidden_x2 = nn.Sequential(
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=2, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.Conv1d(in_channels=4, out_channels=4, kernel_size=5),
            nn.SELU(),
            nn.AvgPool1d(kernel_size=2),
            nn.Conv1d(in_channels=4, out_channels=2, kernel_size=3),
            nn.SELU(),
            nn.AvgPool1d(kernel_size=2),
            nn.Conv1d(in_channels=2, out_channels=2, kernel_size=3),
            nn.SELU(),
            nn.AvgPool1d(kernel_size=2),
        )

        # Flatten layer
        self.flatten_layer = nn.Flatten()

        # Final embedding block - Output 4 values - linear
        self.hidden_embedding = nn.Sequential(
            nn.Linear(26, 16),
            nn.SELU(),
            nn.Linear(16, 8),
            nn.SELU(),
            nn.Linear(8, 4),
        )

    def forward(self, x, n=-1):
        x = torch.swapaxes(x, 1, 2)  # output shape - samples, (real, imag), frequency
        x = self.hidden_x1(x)
        xfc = torch.reshape(x, (n, 256))  # batch size, features
        xfc = self.hidden_xfc(xfc)
        x = torch.reshape(x, (n, 2, 128))  # batch size, (real, imag), timesteps
        x = self.hidden_x2(x)
        cnn_flat = self.flatten_layer(x)
        encoded = torch.cat((cnn_flat, xfc), 1)  # merge dense and 1d conv.
        embedding = self.hidden_embedding(encoded)  # output is 4 parameters

        # corrects the scaling of the parameters
        unscaled_param = (
            embedding * torch.tensor(params_scaler.var_[0:4] ** 0.5).cuda()
            + torch.tensor(params_scaler.mean_[0:4]).cuda()
        )

        # passes to the pytorch fitting function
        fits = SHO_fit_func_torch(unscaled_param, wvec_freq, device="cuda")

        # extract and return real and imaginary
        real = torch.real(fits)
        real_scaled = (real - torch.tensor(scaler_real.mean).cuda()) / torch.tensor(
            scaler_real.std
        ).cuda()
        imag = torch.imag(fits)
        imag_scaled = (imag - torch.tensor(scaler_imag.mean).cuda()) / torch.tensor(
            scaler_imag.std
        ).cuda()
        out = torch.stack((real_scaled, imag_scaled), 2)
        return out

## Trains NN

### Train/Test Split

In [ ]:
data_train, data_test, params_train, params_test = train_test_split(
    scaled_data, scaled_params, test_size=0.7, random_state=42
)

params_test_unscaled = params_scaler.inverse_transform(params_test)

### Training

In [ ]:
if not use_pre_trained_models:
  max_num_of_updates = scaled_data.shape[0] // 16 * 10
  output_tensor_dict = {}
  seed = 0
  noise_reshaped = resample(noise.reshape(-1, 165), 80, axis=1)
  noise_2d = np.stack((np.real(noise_reshaped), np.imag(noise_reshaped)), axis=2)
  noise_train, noise_test = train_test_split(noise_2d, test_size=0.7, random_state=42)

In [ ]:
if not use_pre_trained_models:
  for noise_factor in noise_levels:
      noise_factor = float(noise_factor)
      random_seed(seed=42)
      torch.cuda.empty_cache()
      model = SHO_Model().cuda()

      loss_func = torch.nn.MSELoss()
      batch_size = 128
      data_noise = scaled_data + noise_2d * noise_factor
      data_train_noise = data_train + noise_train * noise_factor
      data_test_noise = data_test + noise_test * noise_factor

      optimizer = torch.optim.Adam(model.parameters())
      train_dataloader = DataLoader(data_train_noise, batch_size=batch_size)
      epochs = max_num_of_updates * batch_size // scaled_data.shape[0] // 16
      print(f"Training with batch size = {batch_size}, noise factor = {noise_factor}")
      start_time_training = time.time()
      for epoch in range(epochs):
          start_time = time.time()
          train_loss = 0.
          total_num = 0

          model.train()

          for train_batch in train_dataloader:
              pred = model(train_batch.double().cuda())
              optimizer.zero_grad()
              loss = loss_func(train_batch.double().cuda(), pred)
              loss.backward(create_graph=True)
              train_loss += loss.item() * pred.shape[0]
              total_num += pred.shape[0]
              optimizer.step()

          train_loss /= total_num
          torch.save(model.state_dict(), f'./Trained Models/SHO Fitter/model_noise_{noise_factor}_bs128.pt')

          print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, train_loss))
          print("--- %s seconds ---" % (time.time() - start_time))

      print(f"Training with batch size={batch_size} took {time.time() - start_time_training} seconds\n")

      del train_dataloader
      del data_noise
      del data_train_noise
      del data_test_noise
      del model
      gc.collect()
      torch.cuda.empty_cache()

### Computes fits

In [ ]:
for noise_level in noise_levels:
    SHO_Fitter(f'data_file_noise_{noise_level}.h5', force=True)
    print(f'Computation for noise level={noise_level} is done!')

### Uses Pre-computed Noisy SHO Fits

In [ ]:
# TODO: download from Zenodo

In [ ]:
if use_pre_computed_vars:
  !unzip noisy_sho_fits.zip